In [1]:
!pip install pyspark

     |████████████████████████████████| 212.3MB 67kB/s 
     |████████████████████████████████| 204kB 42.4MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=b3cc3c79392bb7800dbe576eac50e2ed0a7537ddd2453cdf45f78206ec5901ac
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('ML project').getOrCreate()

In [14]:
# load data 
df = spark.read.csv('iris.data', header = True, inferSchema = True)
df.printSchema()

root
 |-- 5.1: double (nullable = true)
 |-- 3.5: double (nullable = true)
 |-- 1.4: double (nullable = true)
 |-- 0.2: double (nullable = true)
 |-- Iris-setosa: string (nullable = true)



In [15]:
k=len(df.columns)
for i in range(k):
  old_name = df.columns[i]
  new_name = 'f'+str(i)
  print(old_name,new_name)
  df=df.withColumnRenamed(old_name,new_name)

5.1 f0
3.5 f1
1.4 f2
0.2 f3
Iris-setosa f4


In [16]:
df.show()

+---+---+---+---+-----------+
| f0| f1| f2| f3|         f4|
+---+---+---+---+-----------+
|4.9|3.0|1.4|0.2|Iris-setosa|
|4.7|3.2|1.3|0.2|Iris-setosa|
|4.6|3.1|1.5|0.2|Iris-setosa|
|5.0|3.6|1.4|0.2|Iris-setosa|
|5.4|3.9|1.7|0.4|Iris-setosa|
|4.6|3.4|1.4|0.3|Iris-setosa|
|5.0|3.4|1.5|0.2|Iris-setosa|
|4.4|2.9|1.4|0.2|Iris-setosa|
|4.9|3.1|1.5|0.1|Iris-setosa|
|5.4|3.7|1.5|0.2|Iris-setosa|
|4.8|3.4|1.6|0.2|Iris-setosa|
|4.8|3.0|1.4|0.1|Iris-setosa|
|4.3|3.0|1.1|0.1|Iris-setosa|
|5.8|4.0|1.2|0.2|Iris-setosa|
|5.7|4.4|1.5|0.4|Iris-setosa|
|5.4|3.9|1.3|0.4|Iris-setosa|
|5.1|3.5|1.4|0.3|Iris-setosa|
|5.7|3.8|1.7|0.3|Iris-setosa|
|5.1|3.8|1.5|0.3|Iris-setosa|
|5.4|3.4|1.7|0.2|Iris-setosa|
+---+---+---+---+-----------+
only showing top 20 rows



In [18]:
# Show Dataset in DataFrame
import pandas as pd
pd.DataFrame(df.take(5), columns=df.columns).transpose()

,0,1,2,3,4
f0,4.9,4.7,4.6,5,5.4
f1,3,3.2,3.1,3.6,3.9
f2,1.4,1.3,1.5,1.4,1.7
f3,0.2,0.2,0.2,0.2,0.4
f4,Iris-setosa,Iris-setosa,Iris-setosa,Iris-setosa,Iris-setosa


In [19]:
# Data grouping by class 
class_name = df.columns[len(df.columns)-1]
df.groupby(class_name).count().toPandas()

,f4,count
0,Iris-virginica,50
1,Iris-setosa,49
2,Iris-versicolor,50


In [21]:
# convert string to numeric 
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer()
indexer.setInputCol(class_name).setOutputCol("label")
df1 = indexer.fit(df).transform(df)


In [22]:
df1.show()

+---+---+---+---+-----------+-----+
| f0| f1| f2| f3|         f4|label|
+---+---+---+---+-----------+-----+
|4.9|3.0|1.4|0.2|Iris-setosa|  2.0|
|4.7|3.2|1.3|0.2|Iris-setosa|  2.0|
|4.6|3.1|1.5|0.2|Iris-setosa|  2.0|
|5.0|3.6|1.4|0.2|Iris-setosa|  2.0|
|5.4|3.9|1.7|0.4|Iris-setosa|  2.0|
|4.6|3.4|1.4|0.3|Iris-setosa|  2.0|
|5.0|3.4|1.5|0.2|Iris-setosa|  2.0|
|4.4|2.9|1.4|0.2|Iris-setosa|  2.0|
|4.9|3.1|1.5|0.1|Iris-setosa|  2.0|
|5.4|3.7|1.5|0.2|Iris-setosa|  2.0|
|4.8|3.4|1.6|0.2|Iris-setosa|  2.0|
|4.8|3.0|1.4|0.1|Iris-setosa|  2.0|
|4.3|3.0|1.1|0.1|Iris-setosa|  2.0|
|5.8|4.0|1.2|0.2|Iris-setosa|  2.0|
|5.7|4.4|1.5|0.4|Iris-setosa|  2.0|
|5.4|3.9|1.3|0.4|Iris-setosa|  2.0|
|5.1|3.5|1.4|0.3|Iris-setosa|  2.0|
|5.7|3.8|1.7|0.3|Iris-setosa|  2.0|
|5.1|3.8|1.5|0.3|Iris-setosa|  2.0|
|5.4|3.4|1.7|0.2|Iris-setosa|  2.0|
+---+---+---+---+-----------+-----+
only showing top 20 rows



In [23]:
# Split the data
(training_data, test_data) = df1.randomSplit([0.8,0.2])

In [24]:
training_data.toPandas()

,f0,f1,f2,f3,f4,label
0,4.3,3.0,1.1,0.1,Iris-setosa,2.0
1,4.4,2.9,1.4,0.2,Iris-setosa,2.0
2,4.4,3.0,1.3,0.2,Iris-setosa,2.0
3,4.4,3.2,1.3,0.2,Iris-setosa,2.0
4,4.5,2.3,1.3,0.3,Iris-setosa,2.0
...,...,...,...,...,...,...
114,7.4,2.8,6.1,1.9,Iris-virginica,1.0
115,7.6,3.0,6.6,2.1,Iris-virginica,1.0
116,7.7,2.6,6.9,2.3,Iris-virginica,1.0
117,7.7,2.8,6.7,2.0,Iris-virginica,1.0


In [25]:
# features and class 
class_name = 'label'
feature_names = df.columns[:-1]
print(class_name)
print(feature_names)

label
['f0', 'f1', 'f2', 'f3']


In [26]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler()
assembler.setInputCols(feature_names).setOutputCol('features')

transformed_data = assembler.transform(df1)

In [27]:
transformed_data.show()

+---+---+---+---+-----------+-----+-----------------+
| f0| f1| f2| f3|         f4|label|         features|
+---+---+---+---+-----------+-----+-----------------+
|4.9|3.0|1.4|0.2|Iris-setosa|  2.0|[4.9,3.0,1.4,0.2]|
|4.7|3.2|1.3|0.2|Iris-setosa|  2.0|[4.7,3.2,1.3,0.2]|
|4.6|3.1|1.5|0.2|Iris-setosa|  2.0|[4.6,3.1,1.5,0.2]|
|5.0|3.6|1.4|0.2|Iris-setosa|  2.0|[5.0,3.6,1.4,0.2]|
|5.4|3.9|1.7|0.4|Iris-setosa|  2.0|[5.4,3.9,1.7,0.4]|
|4.6|3.4|1.4|0.3|Iris-setosa|  2.0|[4.6,3.4,1.4,0.3]|
|5.0|3.4|1.5|0.2|Iris-setosa|  2.0|[5.0,3.4,1.5,0.2]|
|4.4|2.9|1.4|0.2|Iris-setosa|  2.0|[4.4,2.9,1.4,0.2]|
|4.9|3.1|1.5|0.1|Iris-setosa|  2.0|[4.9,3.1,1.5,0.1]|
|5.4|3.7|1.5|0.2|Iris-setosa|  2.0|[5.4,3.7,1.5,0.2]|
|4.8|3.4|1.6|0.2|Iris-setosa|  2.0|[4.8,3.4,1.6,0.2]|
|4.8|3.0|1.4|0.1|Iris-setosa|  2.0|[4.8,3.0,1.4,0.1]|
|4.3|3.0|1.1|0.1|Iris-setosa|  2.0|[4.3,3.0,1.1,0.1]|
|5.8|4.0|1.2|0.2|Iris-setosa|  2.0|[5.8,4.0,1.2,0.2]|
|5.7|4.4|1.5|0.4|Iris-setosa|  2.0|[5.7,4.4,1.5,0.4]|
|5.4|3.9|1.3|0.4|Iris-setosa

In [28]:
# Split the data
(training_data, test_data) = transformed_data.randomSplit([0.8,0.2], seed = 51800430)
print("Training Dataset Count: " + str(training_data.count()))
print("Test Dataset Count: " + str(test_data.count()))

Training Dataset Count: 119
Test Dataset Count: 30


In [29]:
training_data.toPandas()

,f0,f1,f2,f3,f4,label,features
0,4.4,2.9,1.4,0.2,Iris-setosa,2.0,"[4.4, 2.9, 1.4, 0.2]"
1,4.4,3.0,1.3,0.2,Iris-setosa,2.0,"[4.4, 3.0, 1.3, 0.2]"
2,4.5,2.3,1.3,0.3,Iris-setosa,2.0,"[4.5, 2.3, 1.3, 0.3]"
3,4.6,3.1,1.5,0.2,Iris-setosa,2.0,"[4.6, 3.1, 1.5, 0.2]"
4,4.6,3.2,1.4,0.2,Iris-setosa,2.0,"[4.6, 3.2, 1.4, 0.2]"
...,...,...,...,...,...,...,...
114,7.7,2.6,6.9,2.3,Iris-virginica,1.0,"[7.7, 2.6, 6.9, 2.3]"
115,7.7,2.8,6.7,2.0,Iris-virginica,1.0,"[7.7, 2.8, 6.7, 2.0]"
116,7.7,3.0,6.1,2.3,Iris-virginica,1.0,"[7.7, 3.0, 6.1, 2.3]"
117,7.7,3.8,6.7,2.2,Iris-virginica,1.0,"[7.7, 3.8, 6.7, 2.2]"


In [30]:
# Define the model
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(labelCol='label', 
                            featuresCol='features',
                            maxDepth=5)

In [31]:
# Fit the model
model = rf.fit(training_data)

In [32]:
# Predict with the test dataset
rf_predictions = model.transform(test_data)

In [33]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

multi_evaluator = MulticlassClassificationEvaluator(labelCol = 'label', metricName = 'accuracy')
print('Random Forest classifier Accuracy:', multi_evaluator.evaluate(rf_predictions))

Random Forest classifier Accuracy: 0.9666666666666667


In [35]:
from pyspark.ml.classification import DecisionTreeClassifier

dt = DecisionTreeClassifier(featuresCol = 'features', labelCol = 'label', maxDepth = 3)
dtModel = dt.fit(training_data)
dt_predictions = dtModel.transform(test_data)
dt_predictions.show(10)

+---+---+---+---+---------------+-----+-----------------+--------------+-------------+----------+
| f0| f1| f2| f3|             f4|label|         features| rawPrediction|  probability|prediction|
+---+---+---+---+---------------+-----+-----------------+--------------+-------------+----------+
|4.3|3.0|1.1|0.1|    Iris-setosa|  2.0|[4.3,3.0,1.1,0.1]|[0.0,0.0,39.0]|[0.0,0.0,1.0]|       2.0|
|4.4|3.2|1.3|0.2|    Iris-setosa|  2.0|[4.4,3.2,1.3,0.2]|[0.0,0.0,39.0]|[0.0,0.0,1.0]|       2.0|
|4.6|3.6|1.0|0.2|    Iris-setosa|  2.0|[4.6,3.6,1.0,0.2]|[0.0,0.0,39.0]|[0.0,0.0,1.0]|       2.0|
|4.9|2.5|4.5|1.7| Iris-virginica|  1.0|[4.9,2.5,4.5,1.7]|[37.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|4.9|3.1|1.5|0.1|    Iris-setosa|  2.0|[4.9,3.1,1.5,0.1]|[0.0,0.0,39.0]|[0.0,0.0,1.0]|       2.0|
|4.9|3.1|1.5|0.1|    Iris-setosa|  2.0|[4.9,3.1,1.5,0.1]|[0.0,0.0,39.0]|[0.0,0.0,1.0]|       2.0|
|4.9|3.1|1.5|0.1|    Iris-setosa|  2.0|[4.9,3.1,1.5,0.1]|[0.0,0.0,39.0]|[0.0,0.0,1.0]|       2.0|
|5.0|2.0|3.5|1.0|Iri

In [36]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

multi_evaluator = MulticlassClassificationEvaluator(labelCol = 'label', metricName = 'accuracy')
print('Decision Tree Accuracy:', multi_evaluator.evaluate(dt_predictions))

Decision Tree Accuracy: 0.9666666666666667


In [37]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol = 'features', labelCol = 'label', maxIter=10)
lrModel = lr.fit(training_data)
lr_predictions = dtModel.transform(test_data)

In [38]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

multi_evaluator = MulticlassClassificationEvaluator(labelCol = 'label', metricName = 'accuracy')
print('Logistic Regression Accuracy:', multi_evaluator.evaluate(lr_predictions))

Logistic Regression Accuracy: 0.9666666666666667
